In [ ]:
import codebase.match_data as match
import codebase.settings as settings
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib as plt
import seaborn as sns
import codebase.analysis_functions as af
import codebase.web_scrape_functions as wsf
from utils import logger

m  = match.MatchData('1263466')
%load_ext autoreload
%autoreload 2

In [ ]:
top_players = wsf.read_statsguru('https://stats.espncricinfo.com/ci/engine/stats/index.html?class=1;filter=advanced;orderby=batting_average;qualmin1=30;qualval1=matches;template=results;type=batting', table_name='Overall figures')
top_players = pd.DataFrame(top_players[0])
top_players.head()
top_players.Player = [wsf.player_id_from_link(player, playername=False)[1] for player in top_players.Player]
top_players.head()
match_list = []
for player in top_players.Player:
    match_list += wsf.player_match_list(player)

set([int(id) for id in match_list])
len(match_list)

In [ ]:
import os

for _match in match_list:
    if os.path.exists(os.path.join("D:\Documents\Cricket\data_analysis\cricket_data_analysis\data\match_jsons", f"{_match}_match_jsons.json")):
        logger.info("Skipping as match %s already exists", _match)
        continue
    match.MatchData(match_id=_match)

In [ ]:
PLAYER_ID = '49496'

In [ ]:
cw_bowl = af.get_player_contribution(PLAYER_ID, m, 'bowl')
cw_bowl.sum()

In [ ]:
player_map = {int(player['player_id']):player['card_long'] for player in m.all_players}
pprint(m.__dict__.keys())

In [ ]:
fifth_ashes_test = pd.DataFrame.from_dict(m.get_full_comms())
fifth_ashes_test = fifth_ashes_test.dropna(how='all', axis=1)
fifth_ashes_test.keys()

In [ ]:
fifth_ashes_test.head()

In [ ]:
all_fours = fifth_ashes_test[fifth_ashes_test.isFour == True]


In [ ]:
all_fours["batsman_name"] = all_fours["batsmanPlayerId"].map(player_map)
all_fours.batsman_name.value_counts()

In [ ]:
sns.set_theme(style='darkgrid')
df = pd.DataFrame(all_fours.batsman_name.value_counts()).rename(columns={'batsman_name': 'boundary_counts'})
df['batsman'] = df.index
df.reset_index(drop=True, inplace=True)
fig_dims = (15,10)
fig,ax = plt.pyplot.subplots(figsize=fig_dims)
bar = sns.barplot(data=df, x='batsman', y='boundary_counts', palette='Blues_d', ax=ax)
x_labels = bar.get_xticklabels();
bar.set_xticklabels(x_labels, rotation=90);

In [ ]:
af.get_player_map(m)
wickets = af.get_balls_event(fifth_ashes_test, 'isWicket', True)
af.map_players(m, wickets)
wickets_df = af.series_to_df(wickets.bowlerName.value_counts(), ['bowlerName', 'Wickets'])
af.graph_seaborn_barplot(wickets_df, 'bowlerName', 'Wickets');

In [ ]:
af.get_aggregates(m, 'wickets')

In [ ]:
af.get_aggregates(m, 'bat-fours')

In [ ]:
af.get_aggregates(m, 'bowl-sixes')

In [ ]:
event_df = af.get_aggregates(m, 'bat-runs')
event_df

In [ ]:
af.get_aggregates(m, 'bowl-runs')

In [ ]:
test_comms = fifth_ashes_test.copy()
af.process_text_comms(test_comms)
test_comms.head()

In [ ]:
print(test_comms.iloc[0,24])
af.remove_html(test_comms.iloc[0,24])

In [ ]:
print(test_comms.iloc[0,2])

In [ ]:
comment_example = test_comms.iloc[0,2]

In [ ]:
type(comment_example[0])

In [ ]:
html_coms = test_comms.iloc[:,2].to_frame()
html_coms['type'] = html_coms['commentTextItems'].map(type)

In [ ]:
html_coms = html_coms[html_coms['type'] != html_coms.iloc[0,1]]
html_coms

In [ ]:
test_comms = af.pre_transform_comms(m)
test_comms.head()